In [17]:
#imports 
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
import gc
from sklearn.linear_model import LinearRegression
import random
import datetime as dt

In [18]:
XGB_WEIGHT = 0.6415
BASELINE_WEIGHT = 0.0056
OLS_WEIGHT = 0.0828
XGB1_WEIGHT = 0.8083
BASELINE_PRED = 0.0115

In [19]:
prop = pd.read_csv('C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\properties_2017.csv', low_memory=False)
train = pd.read_csv("C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\train.csv")

In [20]:
################
################
##  LightGBM  ##
################
################

##### PROCESS DATA FOR LIGHTGBM
print( "\nProcessing data for LightGBM ..." )
for c, dtype in zip(prop.columns, prop.dtypes):	
    if dtype == np.float64:		
        prop[c] = prop[c].astype(np.float32)


Processing data for LightGBM ...


In [21]:
df_train = train.merge(prop, how='left', on='parcelid')
df_train.fillna(df_train.median(),inplace = True)

,parcelid,logerror,transactiondate,subdate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.027600,1/1/2016,201601,1.0,7.0,585.0,2.0,3.0,4.0,...,1.0,1,218000.0,545000.0,2016.0,327000.0,6735.879883,NaN,14.0,6.037107e+13
1,14366692,-0.168400,1/1/2016,201601,1.0,7.0,585.0,3.5,4.0,4.0,...,1.0,1,346458.0,974900.0,2016.0,628442.0,10153.019531,NaN,14.0,6.037603e+13
2,12098116,-0.004000,1/1/2016,201601,1.0,7.0,585.0,3.0,2.0,4.0,...,1.0,1,197900.0,989500.0,2016.0,791600.0,11484.480469,NaN,14.0,6.037464e+13
3,12643413,0.021800,2/1/2016,201601,1.0,7.0,585.0,2.0,2.0,4.0,...,1.0,1,174133.0,248613.0,2016.0,74480.0,3048.739990,NaN,14.0,6.037296e+13
4,14432541,-0.005000,2/1/2016,201601,1.0,7.0,585.0,2.5,4.0,4.0,...,2.0,1,172160.0,441177.0,2016.0,269017.0,5488.959961,NaN,14.0,6.059042e+13
5,11509835,-0.270500,2/1/2016,201601,1.0,7.0,585.0,4.0,4.0,4.0,...,1.0,1,894079.0,2485281.0,2016.0,1591202.0,27126.570312,NaN,14.0,6.037621e+13
6,12286022,0.044000,2/1/2016,201601,1.0,7.0,585.0,1.0,2.0,4.0,...,1.0,1,65533.0,113221.0,2016.0,47688.0,2304.969971,NaN,14.0,6.037542e+13
7,17177301,0.163800,2/1/2016,201601,1.0,7.0,585.0,2.5,3.0,4.0,...,2.0,1,113200.0,323864.0,2016.0,210664.0,3946.919922,NaN,14.0,6.111003e+13
8,14739064,-0.003000,2/1/2016,201601,1.0,7.0,585.0,1.0,2.0,4.0,...,1.0,1,67853.0,213267.0,2016.0,145414.0,2172.879883,NaN,14.0,6.059042e+13
9,14677559,0.084300,3/1/2016,201601,1.0,7.0,585.0,2.0,2.0,4.0,...,1.0,1,111654.0,193872.0,2016.0,82218.0,1940.260010,NaN,14.0,6.059063e+13


In [22]:
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'subdate', 
                         'propertycountylandusecode', 'fireplacecnt', 'fireplaceflag'], axis=1)
#x_train['Ratio_1'] = x_train['taxvaluedollarcnt']/x_train['taxamount']
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)


train_columns = x_train.columns

(113039, 53) (113039,)


In [23]:
for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

1985

In [24]:
x_train = x_train.values.astype(np.float32, copy=False)
d_train = lgb.Dataset(x_train, label=y_train)

In [25]:
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.0021 # shrinkage_rate
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'          # or 'mae'
params['sub_feature'] = 0.345    
params['bagging_fraction'] = 0.85 # sub_row
params['bagging_freq'] = 40
params['num_leaves'] = 512        # num_leaf
params['min_data'] = 500         # min_data_in_leaf
params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
params['verbose'] = 0
params['feature_fraction_seed'] = 2
params['bagging_seed'] = 3

In [26]:
print("\nFitting LightGBM model ...")
clf = lgb.train(params, d_train, 430)

del d_train; gc.collect()
del x_train; gc.collect()


Fitting LightGBM model ...


0

In [28]:
print("\nPrepare for LightGBM prediction ...")
print("   Read sample file ...")
sample = pd.read_csv("C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\sample_submission.csv")
sample.columns = ['parcelid','201704','201705','201706','201707','201708','201709']


Prepare for LightGBM prediction ...
   Read sample file ...


In [29]:
print("   Merge with property data ...")
df_test = sample.merge(prop, on='parcelid', how='left')

   Merge with property data ...


In [30]:
del sample, prop; gc.collect()

842

In [31]:
x_test = df_test[train_columns]
print("   ...")
del df_test; gc.collect()
print("   Preparing x_test...")
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
print("   ...")
x_test = x_test.values.astype(np.float32, copy=False)
print("Test shape :", x_test.shape)

print("\nStart LightGBM prediction ...")
p_test = clf.predict(x_test)

del x_test; gc.collect()

print( "\nUnadjusted LightGBM predictions:" )
print( pd.DataFrame(p_test).head() )

   ...
   Preparing x_test...
   ...
Test shape : (2985217, 53)

Start LightGBM prediction ...

Unadjusted LightGBM predictions:
          0
0  0.053559
1  0.054325
2  0.012344
3  0.010245
4  0.009351


In [32]:
properties = pd.read_csv('C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\properties_2017.csv', low_memory=False)

In [33]:
print( "\nProcessing data for XGBoost ...")
for c in properties.columns:
    properties[c]=properties[c].fillna(-1)
    if properties[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        properties[c] = lbl.transform(list(properties[c].values))


Processing data for XGBoost ...


In [34]:
train_df = train.merge(properties, how='left', on='parcelid')

In [36]:
x_train = train_df.drop(['parcelid', 'logerror','transactiondate','subdate'], axis=1)

In [37]:
x_test = properties.drop(['parcelid'], axis=1)

In [38]:
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))

Shape train: (113039, 57)
Shape test: (2985217, 57)


In [39]:
train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.419 ]



In [40]:
x_train = train_df.drop(['parcelid', 'logerror','transactiondate','subdate'], axis=1)

In [41]:
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)

In [42]:
print('After removing outliers:')     
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))

After removing outliers:
Shape train: (110749, 57)
Shape test: (2985217, 57)


In [43]:
print("\nSetting up data for XGBoost ...")
# xgboost params
xgb_params = {
    'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 0.8,   
    'alpha': 0.4, 
    'base_score': y_mean,
    'silent': 1
}


Setting up data for XGBoost ...


In [44]:
dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

In [45]:
num_boost_rounds = 250
print("num_boost_rounds="+str(num_boost_rounds))

# train model
print( "\nTraining XGBoost ...")
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

print( "\nPredicting with XGBoost ...")
xgb_pred1 = model.predict(dtest)

print( "\nFirst XGBoost predictions:" )
print( pd.DataFrame(xgb_pred1).head() )

num_boost_rounds=250

Training XGBoost ...

Predicting with XGBoost ...

First XGBoost predictions:
          0
0 -0.001644
1 -0.039085
2  0.040538
3  0.033255
4 -0.013371


In [46]:
print("\nSetting up data for XGBoost ...")
# xgboost params
xgb_params = {
    'eta': 0.033,
    'max_depth': 6,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': y_mean,
    'silent': 1
}
num_boost_rounds = 150
print("num_boost_rounds="+str(num_boost_rounds))

print( "\nTraining XGBoost again ...")
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

print( "\nPredicting with XGBoost again ...")
xgb_pred2 = model.predict(dtest)

print( "\nSecond XGBoost predictions:" )
print( pd.DataFrame(xgb_pred2).head() )


Setting up data for XGBoost ...
num_boost_rounds=150

Training XGBoost again ...

Predicting with XGBoost again ...

Second XGBoost predictions:
          0
0  0.033204
1 -0.005585
2  0.034313
3  0.044139
4  0.014955


In [47]:
xgb_pred = XGB1_WEIGHT*xgb_pred1 + (1-XGB1_WEIGHT)*xgb_pred2
print( "\nCombined XGBoost predictions:" )
print( pd.DataFrame(xgb_pred).head() )
del train_df
del x_train
del x_test
del properties
del dtest
del dtrain
del xgb_pred1
del xgb_pred2 
gc.collect()


Combined XGBoost predictions:
          0
0  0.005036
1 -0.032663
2  0.039344
3  0.035341
4 -0.007941


179

In [48]:
# This section is derived from the1owl's notebook:
#    https://www.kaggle.com/the1owl/primer-for-the-zillow-pred-approach
# which I (Andy Harless) updated and made into a script:
#    https://www.kaggle.com/aharless/updated-script-version-of-the1owl-s-basic-ols
np.random.seed(7)
random.seed(7)

In [49]:
train = pd.read_csv("C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\train.csv", parse_dates=["transactiondate"])


In [50]:
train.head()

,parcelid,logerror,transactiondate,subdate
0,11016594,0.0276,2016-01-01,201601
1,14366692,-0.1684,2016-01-01,201601
2,12098116,-0.0040,2016-01-01,201601
3,12643413,0.0218,2016-02-01,201601
4,14432541,-0.0050,2016-02-01,201601


In [51]:
properties = pd.read_csv('C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\properties_2017.csv', low_memory=False)


In [52]:
submission = pd.read_csv("C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\sample_submission.csv")
submission.columns = ['parcelid','201704','201705','201706','201707','201708','201709']

In [53]:
print(len(train),len(properties),len(submission))

113039 2985217 2985217


In [54]:
def get_features(df):
    df["transactiondate"] = pd.to_datetime(df["transactiondate"])
    df["transactiondate_year"] = df["transactiondate"].dt.year
    df["transactiondate_month"] = df["transactiondate"].dt.month
    df['transactiondate'] = df['transactiondate'].dt.quarter
    df = df.fillna(-1.0)
    return df

def MAE(y, ypred):
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)

In [55]:
train = pd.merge(train, properties, how='left', on='parcelid')

In [56]:
train.head()

,parcelid,logerror,transactiondate,subdate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,201601,1.0,NaN,NaN,2.0,3.0,NaN,...,NaN,NaN,218000.0,545000.0,2016.0,327000.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,201601,NaN,NaN,NaN,3.5,4.0,NaN,...,NaN,NaN,346458.0,974900.0,2016.0,628442.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,201601,1.0,NaN,NaN,3.0,2.0,NaN,...,NaN,NaN,197900.0,989500.0,2016.0,791600.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-02-01,201601,1.0,NaN,NaN,2.0,2.0,NaN,...,NaN,NaN,174133.0,248613.0,2016.0,74480.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-02-01,201601,NaN,NaN,NaN,2.5,4.0,NaN,...,2.0,NaN,172160.0,441177.0,2016.0,269017.0,5488.96,NaN,NaN,6.059042e+13


In [57]:
y = train['logerror'].values

In [58]:
properties.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


In [59]:
test = pd.merge(submission, properties, how='left', left_on='parcelid', right_on='parcelid')

In [60]:
properties = [] #memory

In [61]:
test.head()

,parcelid,201704,201705,201706,201707,201708,201709,airconditioningtypeid,architecturalstyletypeid,basementsqft,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,0,0,0,0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,0,0,0,0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,0,0,0,0,0,0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,0,0,0,0,0,0,NaN,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,0,0,0,0,0,0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


In [62]:
exc = [train.columns[c] for c in range(len(train.columns)) if train.dtypes[c] == 'O'] + ['logerror','parcelid','subdate']
col = [c for c in train.columns if c not in exc]

In [63]:
train = get_features(train[col])

C:\Users\Lin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Lin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Lin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [64]:
test['transactiondate'] = '2016-01-01'

In [65]:
test = get_features(test[col])

C:\Users\Lin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Lin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Lin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [66]:
reg = LinearRegression(n_jobs=-1)

In [67]:
reg.fit(train, y); print('fit...')
print(MAE(y, reg.predict(train)))
train = [];  y = [] #memory

fit...
0.0692188156691


In [68]:
test_dates = ['2017-04-01','2017-05-01','2017-06-01','2017-07-01','2017-08-01','2017-09-01']
test_columns = ['201704','201705','201706','201707','201708','201709']

In [69]:
print( "\nCombining XGBoost, LightGBM, and baseline predicitons ..." )
lgb_weight = (1 - XGB_WEIGHT - BASELINE_WEIGHT) / (1 - OLS_WEIGHT)
xgb_weight0 = XGB_WEIGHT / (1 - OLS_WEIGHT)
baseline_weight0 =  BASELINE_WEIGHT / (1 - OLS_WEIGHT)
pred0 = xgb_weight0*xgb_pred + baseline_weight0*BASELINE_PRED + lgb_weight*p_test


Combining XGBoost, LightGBM, and baseline predicitons ...


In [70]:
print( "\nCombined XGB/LGB/baseline predictions:" )
print( pd.DataFrame(pred0).head() )


Combined XGB/LGB/baseline predictions:
          0
0  0.024200
1 -0.001873
2  0.032338
3  0.028730
4 -0.001886


In [72]:
print( "\nPredicting with OLS and combining with XGB/LGB/baseline predicitons: ..." )
for i in range(len(test_dates)):
    test['transactiondate'] = test_dates[i]
    pred = OLS_WEIGHT*reg.predict(get_features(test)) + (1-OLS_WEIGHT)*pred0
    submission[test_columns[i]] = [float(format(x, '.4f')) for x in pred]
    print('predict...', i)


Predicting with OLS and combining with XGB/LGB/baseline predicitons: ...
predict... 0
predict... 1
predict... 2
predict... 3
predict... 4
predict... 5


In [73]:
print( "\nCombined XGB/LGB/baseline/OLS predictions:" )
print( submission.head() )


Combined XGB/LGB/baseline/OLS predictions:
   parcelid  201704  201705  201706  201707  201708  201709
0  10754147  0.0178  0.0178  0.0177  0.0178  0.0178  0.0177
1  10759547 -0.0067 -0.0068 -0.0068 -0.0067 -0.0068 -0.0068
2  10843547  0.0962  0.0961  0.0961  0.0962  0.0961  0.0961
3  10859147  0.0291  0.0290  0.0289  0.0291  0.0290  0.0289
4  10879947  0.0023  0.0022  0.0021  0.0022  0.0022  0.0021


In [74]:
test_path = "C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\test.csv"
test = pd.read_csv(test_path, low_memory=False)

In [75]:
from math import fabs as absolute
test_size = len(test)

In [76]:
total_ae = 0.0
print("Processing ", test_size, "rows")
for index,row in test.iterrows():
    r = submission.loc[submission['parcelid'] == row[0]]
    
    pred_error = r[str(row[3])]
    total_ae += absolute(pred_error - row[1])
    
    if index % 10000 == 0 and index !=0:
        print("Completed: ", index, "rows")
        
print("===Process Completed===")
print("\n Final score:", total_ae/test_size)

Processing  54849 rows
Completed:  10000 rows
Completed:  20000 rows
Completed:  30000 rows
Completed:  40000 rows
Completed:  50000 rows
===Process Completed===

 Final score: 0.07018905647643531
